## Домашнее задание «Функции потерь и оптимизация» обновленное

### Этапы работы:

In [17]:
from sklearn import datasets
import numpy as np
from datetime import datetime

In [2]:
irises = datasets.load_iris()

Смотрим, какие у нас есть ключи в датасете

In [3]:
keys = irises.keys()
print(keys)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


смотрим, какие есть целевые названия ирисов

In [4]:
print(irises.target_names)

['setosa' 'versicolor' 'virginica']


смотрим, что у нас в данных и что это за данные

In [5]:
print(irises.feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [6]:
print(irises.data[:15])
print(len(irises.data))

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]]
150


смотрим, что у нас в таргетах

In [7]:
print(irises.target)
print(len(irises.target))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
150


Не вижу смысла специально удалять лишний класс в самом датасете (преобразовывать сам датасет, тратя на это усилия), потому что проще будет выбрать из датасета данные и целевые классы, сразу же раскидав их в параметры для обучения, что мы и сделаем заранее.

In [8]:
X = []
Y = []
X_origin = irises.data
for index, item in enumerate(irises.target):
    if item == 1 or item == 2:
        X.append(X_origin[index])
        Y.append(item - 1) # нормализуем

X = np.asarray(X)
Y = np.asarray(Y)

2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберите и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберите только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).

In [13]:
class MyLogisticRegression():

    # так как мне не нужно рисовать диаграммы на плоскости, ограничиваясь только двумя признаками,
    # я использую все признаки из X = irises.data
    def theta(self, theta, X):
        return theta[0] + theta[1] * X[:, 0] + theta[2] * X[:, 1] + theta[3] * X[:, 2] + theta[4] * X[:, 3]

    def sigmoid(self, theta):
        return 1./(1 + np.exp(-theta))

    def predict(self, theta, X):
        return self.sigmoid(self.theta(theta, X))

    def loss_function(self, Y, sigmoid):
        return np.sum(np.square(sigmoid - Y)) / (2 * len(sigmoid))
        # return - np.mean(np.log(sigmoid) * Y + np.log(1 - sigmoid) * (1 - Y))

    def predict_grad(self, X, Y, epochs, rate, theta):
        losses = []
        for _ in range(epochs):
            sigmoid = self.predict(theta, X)
            loss = self.loss_function(Y, sigmoid)
            losses.append(loss)
            theta[0] = theta[0] - rate * np.sum(sigmoid - Y) / len(sigmoid)
            theta[1] = theta[1] - rate * np.sum((sigmoid - Y) * X[:, 0])/len(sigmoid)
            theta[2] = theta[2] - rate * np.sum((sigmoid - Y) * X[:, 1])/len(sigmoid)
            theta[3] = theta[3] - rate * np.sum((sigmoid - Y) * X[:, 2])/len(sigmoid)
            theta[4] = theta[4] - rate * np.sum((sigmoid - Y) * X[:, 3])/len(sigmoid)
            # возвращаем итоговые коэффициенты theta, первое и последнее значение loss, чтобы видеть уменьшение функции потерь,
            # а также для метрики качества используем подсчет ошибок предсказания
        return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, Y, theta)]

    # воспользуемся определениями 13 и 14 из источника https://habr.com/ru/post/318970/ для определения
    # квадрата градиента и бегущего среднего, эпсилон и гамма - произвольно подбираемые коэффициенты
    def predict_rmsprop(self, X, Y, epochs, rate, epsilon, gamma, theta):
        # иницируем нулями массивы градиента и бегущего среднего
        grad = np.zeros(5)
        sq_grad = np.zeros(5)
        losses = []
        for _ in range(epochs):
            sigmoid = self.predict(theta, X)
            loss = self.loss_function(Y, sigmoid)
            losses.append(loss)
            grad[0] = np.sum(sigmoid - Y)/len(sigmoid)
            grad[1] = np.sum((sigmoid - Y) * X[:, 0])/len(sigmoid)
            grad[2] = np.sum((sigmoid - Y) * X[:, 1])/len(sigmoid)
            grad[3] = np.sum((sigmoid - Y) * X[:, 2])/len(sigmoid)
            grad[4] = np.sum((sigmoid - Y) * X[:, 3])/len(sigmoid)
            sq_grad = gamma * sq_grad + (1 - gamma)  * grad ** 2
            theta -= rate * grad / np.sqrt(sq_grad + epsilon)
        return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, Y, theta)]

    # воспользуемся определениями 3, 4 и 5 из источника https://habr.com/ru/post/318970/ для определения
    # экспоненциального скользящего среднего, гамма - произвольно подбираемый коэффициент от 0 до 1, обычно близко к 0.9
    def predict_nadam(self, X, Y, epochs, rate, gamma, theta):
        # инициируем нулями массивы экспоненциального скользящего среднего - текущее и предыдущее
        vt = np.zeros(5)
        vt_prev = np.zeros(5)
        losses = []
        for _ in range(epochs):
            sigmoid = self.predict(theta, X)
            loss = self.loss_function(Y, sigmoid)
            losses.append(loss)
            sigmoid = self.predict(theta - gamma * vt_prev, X)
            vt[0] = (gamma * vt_prev[0] + rate * np.sum(sigmoid - Y))/len(sigmoid)
            vt[1] = (gamma * vt_prev[1] + rate * np.sum((sigmoid - Y) * X[:, 0]))/len(sigmoid)
            vt[2] = (gamma * vt_prev[2] + rate * np.sum((sigmoid - Y) * X[:, 1]))/len(sigmoid)
            vt[3] = (gamma * vt_prev[3] + rate * np.sum((sigmoid - Y) * X[:, 2]))/len(sigmoid)
            vt[4] = (gamma * vt_prev[4] + rate * np.sum((sigmoid - Y) * X[:, 3]))/len(sigmoid)
            theta -= vt
            vt_prev = vt
        return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, Y, theta)]

    def count_errors(self, X, Y, theta):
        count = 0
        for index, item in enumerate(np.around(self.predict(theta, X))):
            # print(int(item), Y[index].item())
            if int(item) != Y[index].item():
                count += 1
        return count

In [21]:
#theta = np.random.normal(size=(5,))
#print(theta)
#случайный выбор theta для тестирования
theta = np.asarray([-0.37221469, -0.3897146,   1.11826103, -1.81180707, -1.2799002 ])
rg = MyLogisticRegression()
start_time = datetime.now()
print(rg.predict_grad(X, Y, 50000, 0.001, theta))
print(datetime.now() - start_time)

[array([-1.49356576, -1.85876646, -0.83705399,  2.47340578,  2.08072563]), 0.2499957613795874, 0.02927782663823318, 3]
0:00:02.617712


In [22]:
#не забываем реанимировать исходный массив theta, так как он уже изменен предыдущим методом по ссылке :)
theta = [-0.37221469, -0.3897146,   1.11826103, -1.81180707, -1.2799002 ]
start_time = datetime.now()
print(rg.predict_rmsprop(X, Y, 50000, 0.01, 0.9999, 0.99999, theta))
print(datetime.now() - start_time)

[array([-4.12039006, -3.59019217, -3.55884558,  5.38330486,  6.20968826]), 0.2499957613795874, 0.013747702466034802, 3]
0:00:02.814012


In [23]:
theta = [-0.37221469, -0.3897146,   1.11826103, -1.81180707, -1.2799002 ]
start_time = datetime.now()
print(rg.predict_nadam(X, Y, 50000, 0.01, 0.9, theta))
print(datetime.now() - start_time)

[array([-4.13738611, -3.59664927, -3.56921045,  5.39443819,  6.22876313]), 0.2499957613795874, 0.013728142819197065, 3]
0:00:03.191954


6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

В целом сравнение показывает, что время работы методов при похожих начальных условиях отличаются незначительно, главное - подобрать коэффициенты для скользящего среднего и Нестерова, кол-во ошибок также одинаковое и незначительное (выбрано в качестве метрики качества)